In [ ]:
import time

import numpy.random as npr

from jax import jit, grad
from jax.scipy.special import logsumexp
import jax.numpy as jnp

In [ ]:
#@title
"""Datasets used in examples."""


import array
import gzip
import os
from os import path
import struct
import urllib.request

import numpy as np


_DATA = "/tmp/jax_example_data/"


def _download(url, filename):
  """Download a url to a file in the JAX data temp directory."""
  if not path.exists(_DATA):
    os.makedirs(_DATA)
  out_file = path.join(_DATA, filename)
  if not path.isfile(out_file):
    urllib.request.urlretrieve(url, out_file)
    print(f"downloaded {url} to {_DATA}")


def _partial_flatten(x):
  """Flatten all but the first dimension of an ndarray."""
  return np.reshape(x, (x.shape[0], -1))


def _one_hot(x, k, dtype=np.float32):
  """Create a one-hot encoding of x of size k."""
  return np.array(x[:, None] == np.arange(k), dtype)


def mnist_raw():
  """Download and parse the raw MNIST dataset."""
  # CVDF mirror of http://yann.lecun.com/exdb/mnist/
  base_url = "https://storage.googleapis.com/cvdf-datasets/mnist/"

  def parse_labels(filename):
    with gzip.open(filename, "rb") as fh:
      _ = struct.unpack(">II", fh.read(8))
      return np.array(array.array("B", fh.read()), dtype=np.uint8)

  def parse_images(filename):
    with gzip.open(filename, "rb") as fh:
      _, num_data, rows, cols = struct.unpack(">IIII", fh.read(16))
      return np.array(array.array("B", fh.read()),
                      dtype=np.uint8).reshape(num_data, rows, cols)

  for filename in ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz",
                   "t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz"]:
    _download(base_url + filename, filename)

  train_images = parse_images(path.join(_DATA, "train-images-idx3-ubyte.gz"))
  train_labels = parse_labels(path.join(_DATA, "train-labels-idx1-ubyte.gz"))
  test_images = parse_images(path.join(_DATA, "t10k-images-idx3-ubyte.gz"))
  test_labels = parse_labels(path.join(_DATA, "t10k-labels-idx1-ubyte.gz"))

  return train_images, train_labels, test_images, test_labels


def mnist(permute_train=False):
  """Download, parse and process MNIST data to unit scale and one-hot labels."""
  train_images, train_labels, test_images, test_labels = mnist_raw()

  train_images = _partial_flatten(train_images) / np.float32(255.)
  test_images = _partial_flatten(test_images) / np.float32(255.)
  train_labels = _one_hot(train_labels, 10)
  test_labels = _one_hot(test_labels, 10)

  if permute_train:
    perm = np.random.RandomState(0).permutation(train_images.shape[0])
    train_images = train_images[perm]
    train_labels = train_labels[perm]

  return train_images, train_labels, test_images, test_labels

# Introduction
This is a basic MNIST example using Numpy and JAX. The primary aim here is simplicity and minimal dependencies. JAX is a very sophisticated and deep parallel linear algebra and differentiation library. It is gaining a lot of popularity lately as it enforces very few constraints and so is ideal for quickly trying new ideas! 

For our purposes however we can think of it as a way to get derivatives of numpy functions (and run numpy on the GPU if one is available). Essentially jax has all the same numpy function you may be familiar with but in the background is doing some gradient tracking and gpu parallelization.



# Preliminary Functions
Below we define five functions which are all we need to set up a minimal neural network to solve MNIST. Two have been filled in (init_random_params and update) to help you understand how we are setting up the data and our network. We will now walk you through how to fill in each function!

### Init Random Params (already implemented)
This function takes in the layer sizes of the network, for example $[784,1024,1024,10]$ is a network with two hidden layers of size $1024$ and input dimensions of $784$ (corresponding to the number of pixels in an MNIST digit) and output of $10$ corresponding to the number of possible labels for the digit. The function then loops through this list and creates weights and biases corresponding to each mapping between layers. For example to map from input to the first hidden layer we need a weight matrix ($W_0$) of size $(784,1024)$ with a bias vector ($b_0$) of size $1024$. We then put this in a tuple $(W_0, b_0)$ which is added to a list of layer parameters: $[(W_0, b_0), (W_1, b_1), ...]$. While you don't have to implement this, it is important you understand this structure to implement the predict function.

### Predict
The predict function takes in two arguments. "params" and "inputs". "params" are the parameters of our neural network and consists of a list of layer parameters. So for example "params[0]" gives the parameters for the first layer of the network. Each layers' parameters consist of a tuple "(W,b)" where "W" is the weights and "b" are the biases. Thus if $l_i$ are the neuron activations at layer $i$ then: $l_{i+1} = l_i*W_i + b_i$ where $*$ is the matrix multiplication (use jnp.dot()). Finally we usually use an activation function denoted by $\sigma$ on the activation of each neuron in a layer. In this case we will use jnp.tanh() for simplicity. So to implement the predict function:
* Set the first activations equal to our inputs ($l_0 = x$)
* For all layers in the network (except the last) compute: $l_{i+1} = \sigma(W_i*l_i + b_i)$
* For the last layer just compute: $l_{n} = W_{n-1}*l_{n-1} + b_i$
* Compute and return the log-softmax of the final activation: $\log\left(\frac{\exp{l_{n}[i]}}{\sum_{i=0}^D\exp{l_{n}[i]}}\right) = l_{n}[i] - \log(\sum_{i=0}^D\exp{l_{n}[i]})$

### Loss
The loss function takes two parameters. "params" which is our array of NN parameters and "batch" which is a batch of data (input, labels). The labels are are hot encodings of which number is actually shown in the image. So the shape of the labels is (n,10) where n is the number of data points in the batch. To implement the function:
* Split the batch into separate inputs and labels
* Use your network to predict the labels for the inputs
* Compute and return the loss (also called error) as $E(\theta) = -\frac{1}{n}\sum_{i=0}^{n}\sum_{j=0}^{10}l_{nj} \hat{l}_{nj}$. Here $l_{nj}$ is the ground truth value for the j-th output for the n-th data point and $\hat{l}_{nj}$ is the corresponding prediction. 

### Accuracy
This function is very similar to the loss function expect now instead of using the output of the network as a whole we will just use this output to make a classification of the data. We will then compare the chosen class with ground truth. The difference with loss is that, to minimize the loss we want the network to be as close to perfect as possible. So we want it to output 1 for the correct class and 0 for the incorrect classes. For accuracy all we care about is that the model has the *largest* output for the correct class. Thus, if the network outputs 0.55 for the correct class and 0.05 for all other (after the softmax) then we are satisfied that it classified the data point correctly. To implement the accuracy function:
* Split the batch into separate inputs and labels
* Define the target class as the argmax of the data labels (returns the index of the most probable class according to the labels)
* Use your network to predict the labels for the inputs
* Obtains the argmax of the network output (find the index of the number which the network believes is most probable for the given image)
* Count the number of times the target and predicted indices match and divide by the batch size: $\frac{1}{n}\sum_{i=0}^{n} argmax(l_n) == argmax(\hat{l}_n)$. Here $argmax(l_n)$ is the ground truth correct classification and $argmax(\hat{l}_n)$

### Update (already implemented)
This is the function which actually does gradient descent and updates the networks weights. Just like loss this function takes in the model parameters and a batch of data. To get the gradients of the loss with respect to the network parameters we will us the built in grad() function from jax. This has a strange structure as it looks like its taking in two sets parameters. Thus you first pass in the function you want to differentiate and then pass in the parameters to that function. For example the loss function takes in the parameters (params, batch) and we want to differentiate the loss function (apologies for using the word "parameters" both as model weights and biases and as the variables passed into a function. I hope it is clear from context which I am referring to). Thus to get the gradients we call grad(loss)(params, batch). In the background this will run the loss function on the params and batch as usual but then the gradient will be calculated and returned instead. The gradient which is calculated is for the first function parameter which is passed in. Thus grad(loss)(params, batch) computes $\frac{\partial loss}{\partial params} = \frac{\partial E(\theta)}{\partial \theta}$. The last line of the function then just loops through the list of model parameters and returns $(W_i = W_i - \alpha\frac{\partial E(\theta)}{\partial W_i}, b_i = b_i - \alpha\frac{\partial E(\theta)}{\partial b_i})$ for all weights and biases in the network (it does a gradient descent step where $\alpha$ is the learning rate).


In [ ]:
def init_random_params(scale, layer_sizes, rng=npr.RandomState(0)):
    return [(scale * rng.randn(m, n), scale * rng.randn(n)) # For each layer initialize weights and biases randomly
          for m, n, in zip(layer_sizes[:-1], layer_sizes[1:])] # Biases are set to the size of the next set of neurons and weights are set to size (previous neurons, next neurons) 

def predict(params, inputs):
    activations = inputs # First activations are set equal to our data
    for w, b in params[:-1]: # For each layer's weights and biases use them to
        outputs =  #TODO # compute the next layers activation (multiply by the weights and add the biases)
        activations = #TODO  # apply the tanh function to the activations from the previous line

    final_w, final_b = params[-1] # Get parameters for last layer
    logits = jnp.dot(activations, final_w) + final_b # Compute the last forward step
    return logits - logsumexp(logits, axis=1, keepdims=True) # Apply the log-softmax

def loss(params, batch):
    inputs, targets = #TODO # Load in a batch of data and split into the input and corresponding labels
    preds = #TODO # use our network to make a prediction from the inputs
    return #TODO # compute the loss of the predictions vs the ground truth labels

def accuracy(params, batch):
    inputs, targets = batch # Load in a batch of data and split into the input and corresponding labels
    target_class = #TODO # Get the index of the label for each data point
    predicted_class = #TODO # use our network to make a prediction from the inputs and get its max index
    return #TODO # Count how many times the network's max index matches the label's max index
 
@jit
def update(params, batch):
    grads = grad(loss)(params, batch) # Get the gradients of the loss w.r.t. the network parameters
    return [(w - step_size * dw, b - step_size * db) # For each layer's weights and biases take a step in the 
          for (w, b), (dw, db) in zip(params, grads)] # negative gradient direction

# Main Training Loop
Here we code up the main training. There are three main pieces. Firstly we define some hyper-parameters. Then we load our data and create an asynchronous function to make reading batches easy. For today you can think of an asynchronous function as a function with memory. Thus if we have a for-loop inside of an asynchronous function and are returning *inside* the for-loop thene everytime we call the function it will start the loop from where it left off. This makes it very easy to track batches. In this case "batches" is the variable of our asynchronous function and to get a new batch from the function we just call "yield(batches)". Finally we training with batches of data for a set number of epochs and track the loss.

In [ ]:
# Define the hyper-parameters of the network and training 
layer_sizes = [784, 1024, 1024, 10]
param_scale = 0.1
step_size = 0.001
num_epochs = 30
batch_size = 64

# Load data and split the data into batches
train_images, train_labels, test_images, test_labels = datasets.mnist()
num_train = train_images.shape[0]
num_complete_batches, leftover = divmod(num_train, batch_size)
num_batches = num_complete_batches + bool(leftover)

# Create an asynchronous function to track our batches
def data_stream():
    rng = npr.RandomState(0)
    while True:
        perm = rng.permutation(num_train)
        for i in range(num_batches):
            batch_idx = perm[i * batch_size:(i + 1) * batch_size]
            yield train_images[batch_idx], train_labels[batch_idx]
batches = data_stream()

####### This is the main training loop #######
# initialize parameters
params = init_random_params(param_scale, layer_sizes)

# For a set number of epoch loop over every batch in our dataset and do one gradient descent update for each batch
for epoch in range(num_epochs):
    start_time = time.time() # sometimes its nice to track how long an epoch takes
    for _ in range(num_batches): # loop over all the batches
        params = update(params, next(batches)) # do one gradient descent update step
    epoch_time = time.time() - start_time # compute how long the epoch took. If you run on GPU it should be really quick

    train_acc = accuracy(params, (train_images, train_labels)) # Calculate the network accuracy at the end of an epoch on training data
    test_acc = accuracy(params, (test_images, test_labels)) # Calculate the network accuracy at the end of an epoch on test data
    print(f"Epoch {epoch} in {epoch_time:0.2f} sec")
    print(f"Training set accuracy {train_acc}")
    print(f"Test set accuracy {test_acc}")